In [ ]:
import pandas as pd
import numpy as np
import pickle
import datetime
import math
from scipy import stats
from collections import Counter
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [ ]:
#Times
STATE = "***"
START_OF_RECORDS = datetime.datetime.fromisoformat("2015-01-01")
END_OF_RECORDS = datetime.datetime.fromisoformat("2019-09-18")
BACK_BUFFER = datetime.timedelta(12*30)
BACK_BUFFER_recent = datetime.timedelta(2*30)

pickle_in = open(STATE + "/opioid_naive_info_known","rb")
opioid_naive_info_known = pickle.load(pickle_in)

#Numpy matrix of NDC codes and metadata for opioid drugs, as determined by the CDC
cdc_opioids = pd.read_csv("~/Resources/CDC_Opioids.csv")
cdc_opioids.loc[:,"ABUSE_DETER"] = 0
#Numpy matrix of NDC codes and metadata for opioid drugs with less abuse potential, as determined by the CDC
abuse_deter_opioids = pd.read_csv("Resources/abuse_deterent.csv")
abuse_deter_opioids.loc[:,"ABUSE_DETER"] = 1
opioid_info = pd.concat([abuse_deter_opioids,cdc_opioids])


#NDC converting function
def convert_ndc_eleven(ndcs):
    elevens = []
    for ndc in ndcs:
        if (ndc[4] == '-'):
            elevens.append(int(ndc.replace('-','')))
        elif (ndc[9] == '-'):
            elevens.append(int(ndc.replace('-','0',1).replace('-','')))
        elif (ndc[10] == '-'):
            elevens.append(int(ndc.replace('-','',1).replace('-','0')))
            
    return elevens

#NDC Constants
naltrexone_ndcs = convert_ndc_eleven(pd.read_csv("Resources/naltrexone_ndc.csv").loc[:,'NDC Package Code'])
fluoxetine_ndcs = convert_ndc_eleven(pd.read_csv("Resources/fluoxetine_ndc.csv").loc[:,'NDC Package Code'])
buprenorphine_ndcs = convert_ndc_eleven(pd.read_csv("Resources/buprenorphine_ndc.csv").loc[:,'NDC Package Code'])
bupropion_ndcs = convert_ndc_eleven(pd.read_csv("Resources/bupropion_ndc.csv").loc[:,'NDC Package Code'])
dacomitinib_ndcs = convert_ndc_eleven(pd.read_csv("Resources/dacomitinib_ndc.csv").loc[:,'NDC Package Code'])
paroxetine_ndcs = convert_ndc_eleven(pd.read_csv("Resources/paroxetine_ndc.csv").loc[:,'NDC Package Code'])
quinidine_ndcs = convert_ndc_eleven(pd.read_csv("Resources/quinidine_ndc.csv").loc[:,'NDC Package Code'])

#analgesics
acetaminophen_ndcs = convert_ndc_eleven(pd.read_csv("Resources/acetaminophen_ndcs.csv").loc[:,'NDC Package Code'])
ibuprofen_ndcs = convert_ndc_eleven(pd.read_csv("Resources/ibuprofen_ndcs.csv").loc[:,'NDC Package Code'])
diclofenac_ndcs = convert_ndc_eleven(pd.read_csv("Resources/diclofenac_ndcs.csv").loc[:,'NDC Package Code'])
ketorolac_ndcs = convert_ndc_eleven(pd.read_csv("Resources/ketorolac_ndcs.csv").loc[:,'NDC Package Code'])
naproxen_ndcs = convert_ndc_eleven(pd.read_csv("Resources/naproxen_ndcs.csv").loc[:,'NDC Package Code'])

duloxetine_ndcs = convert_ndc_eleven(pd.read_csv("Resources/duloxetine_ndcs.csv").loc[:,'NDC Package Code'])
milnacipran_ndcs = convert_ndc_eleven(pd.read_csv("Resources/milnacipran_ndcs.csv").loc[:,'NDC Package Code'])
venlafaxin_ndcs = convert_ndc_eleven(pd.read_csv("Resources/venlafaxin_ndcs.csv").loc[:,'NDC Package Code'])
diclofenac_ndcs = convert_ndc_eleven(pd.read_csv("Resources/diclofenac_ndcs.csv").loc[:,'NDC Package Code'])
gabapentin_ndcs = convert_ndc_eleven(pd.read_csv("Resources/gabapentin_ndcs.csv").loc[:,'NDC Package Code'])
pregabalin_ndcs = convert_ndc_eleven(pd.read_csv("Resources/pregabalin_ndcs.csv").loc[:,'NDC Package Code'])
amitriptalin_ndcs = convert_ndc_eleven(pd.read_csv("Resources/amitriptalin_ndcs.csv").loc[:,'NDC Package Code'])
doxipen_ndcs = convert_ndc_eleven(pd.read_csv("Resources/doxipen_ndcs.csv").loc[:,'NDC Package Code'])
imipramine_ndcs = convert_ndc_eleven(pd.read_csv("Resources/imipramine_ndcs.csv").loc[:,'NDC Package Code'])
desipramine_ndcs = convert_ndc_eleven(pd.read_csv("Resources/desipramine_ndcs.csv").loc[:,'NDC Package Code'])

nortryptaline_ndcs = convert_ndc_eleven(pd.read_csv("Resources/nortryptaline_ndcs.csv").loc[:,'NDC Package Code'])
lidocaine_ndcs = convert_ndc_eleven(pd.read_csv("Resources/lidocaine_ndcs.csv").loc[:,'NDC Package Code'])
capsaicin_ndcs = convert_ndc_eleven(pd.read_csv("Resources/capsaicin_ndcs.csv").loc[:,'NDC Package Code'])


In [ ]:
try:
    opioid_rx_mas
except NameError:
    opioid_rx_mas = pd.read_csv(STATE + "/opioid_rx_mas.csv")

mas_to_opioid_rx = {}
for row in opioid_rx_mas.itertuples():
    mas_to_opioid_rx.setdefault(row.MA_NUM, [])
    mas_to_opioid_rx[row.MA_NUM].append((row.CLAIM_ID,row.CLAIM_FROM_DT))

mas_to_opioid_rx = {ma: sorted(mas_to_opioid_rx[ma], key = lambda tup: tup[1]) for ma in mas_to_opioid_rx}

opioid_rx_count = {}
for ma in mas_to_opioid_rx:
    opioid_rx_count[ma] = len(mas_to_opioid_rx[ma])
d = {
    'rx_count': [opioid_rx_count[ma] for ma in opioid_naive_info_known],
    'OUTCOME': [opioid_naive_info_known[ma][0][4] for ma in opioid_naive_info_known]
}

    
pd.DataFrame(d).groupby('OUTCOME').describe()

In [ ]:
date_distribution_chronic = Counter([opioid_naive_info_known[x][0][2].year for x in opioid_naive_info_known if (opioid_naive_info_known[x][0][4] == 1)])
date_distribution_acute = Counter([opioid_naive_info_known[x][0][2].year for x in opioid_naive_info_known if (opioid_naive_info_known[x][0][4] == 0)])

In [ ]:
date_distribution_chronic

In [ ]:
date_distribution_acute

# DX features

In [ ]:
dx_filter = pd.read_csv(STATE + '/dx_filter.csv',usecols = ['CLAIM_ID','DIAG_CD','DIAG_QUALIFY_CD','MA_NUM','OUTCOME'])

In [ ]:
common_filter = pd.read_csv(STATE + '/common_filter.csv',usecols = ['CLAIM_ID','CLAIM_FROM_DT','CLAIM_TYPE_CD'])

In [ ]:
dx_filter = dx_filter.merge(common_filter,how = 'left',left_on = 'CLAIM_ID', right_on = 'CLAIM_ID')

In [ ]:
dx_filter

In [ ]:
common_filter.shape

In [ ]:
dx_filter.shape

In [ ]:
dx_filter.CLAIM_FROM_DT = dx_filter.CLAIM_FROM_DT.map(lambda x: datetime.datetime.fromisoformat(x))

#Filter for diagnosis made before the rx written date
dx_filter_pre_period = dx_filter[[x[0][2] for x in dx_filter.MA_NUM.map(opioid_naive_info_known)] > dx_filter.CLAIM_FROM_DT]
len(dx_filter_pre_period)

print("Diagnosis claims with correct eligibility: {}".format(len(dx_filter_pre_period)))
dx_filter_pre_period.head()

In [ ]:
#Filter for diagnosis made before the rx written date
dx_filter_init = dx_filter[[x[0][2] for x in dx_filter.MA_NUM.map(opioid_naive_info_known)] == dx_filter.CLAIM_FROM_DT]

dx_filter_init = dx_filter_init.loc[:,['CLAIM_ID','DIAG_CD','OUTCOME','MA_NUM']]
len(dx_filter_init)
dx_filter_init.DIAG_CD = dx_filter_init.DIAG_CD.map(lambda x: x[:3])

In [ ]:
dx_filter_init = dx_filter_init.drop_duplicates()

In [ ]:
len(dx_filter_init)

In [ ]:
dx_filter_init.MA_NUM.nunique()

In [ ]:
dx_filter_init.head()

In [ ]:
common_diags = [x[0] for x in Counter(dx_filter_init.DIAG_CD).most_common(100)]

In [ ]:
dx_filter_init_feature = dx_filter_init.loc[dx_filter_init.DIAG_CD.isin(common_diags)]

In [ ]:
dx_filter_init_feature = pd.get_dummies(dx_filter_init_feature, columns = ['DIAG_CD'], prefix = 'init_diag')

In [ ]:
dx_filter_init_feature = dx_filter_init_feature.drop(['CLAIM_ID','OUTCOME'],axis = 1).groupby("MA_NUM").sum()

In [ ]:
dx_filter_init_feature.shape

In [ ]:
dx_filter_init_feature.to_csv(STATE + "/dx_filter_init_feature.csv")

icd10 comorbidity mappings, based on quan_elixhauser

In [ ]:
icd10_comorbidities = {
     "congestive_heart_failure": ["I099","I110","I130","I132","I255","I420","I425","I426","I427","I428","I429","I43","I50","P290"],
     "cardiac_arrhythmia":["I441","I442","I443","I456","I459","I47","I48","I49","R001","R008","T821","Z450","Z950"],
     "valvular_disease":["A520","I05","I06","I07","I08","I091","I098","I34","I35","I36","I37","I38","I39","Q230","Q231","Q232","Q233","Z952","Z953","Z954"],
     "pulmonary_circulation_disorder":["I26","I27","I280","I288","I289",],
     "peripheral vascular_disorder":["I70","I71","I731","I738","I739","I771","I790","I792","K551","K558","K559","Z958","Z959"],
     "hypertension_uncomplicated":["I10"],
     "hypertension_complicated":["I11","I12","I13","I15"],
     "paralysis":["G041","G114","G801","G802","G81","G82","G830","G831","G832","G833","G834","G839"],
     "other_neurological_disorder":["G10","G11","G12","G13","G20","G21","G22","G254","G255","G312","G318","G319","G32","G35","G36","G37","G40","G41","G931","G934","R470","R56"],
     "chronic_pulmonary_disease":["I278","I279","J40","J41","J42","J43","J44","J45","J46","J47","J60","J61","J62","J63","J64","J65","J66","J67","J684","J701","J703"],
     "diabetes_uncomplicated":["E100","E101","E109","E110","E111","E119","E120","E121","E129","E130","E131","E139","E140","E141","E149"],
     "diabetes_complicated":["E102","E103","E104","E105","E106","E107","E108","E112","E113","E114","E115","E116","E117","E118","E122","E123","E124","E125","E126","E127","E128","E132","E133","E134","E135","E136","E137","E138","E142","E143","E144","E145","E146","E147","E148"],
     "hypothyroidism":["E00","E01","E02","E03","E890"],
     "renal_failure":["I120","I131","N18","N19","N250","Z490","Z491","Z492","Z940","Z992"],
     "liver_disease":["B18","I85","I864","I982","K70","K711","K713","K714","K715","K717","K72","K73","K74","K760","K762","K763","K764","K765","K766","K767","K768","K769","Z944"],
     "gastric_ulcer":["K257","K259","K267","K269","K277","K279","K287","K289"],
     "aids_hiv":["B20","B21","B22","B24"],
     "lymphoma":["C81","C82","C83","C84","C85","C88","C96","C900","C902"],
     "metastatic_cancer":["C77","C78","C79","C80"],
     "solid_tumor_wo_metastasis":["C00","C01","C02","C03","C04","C05","C06","C07","C08","C09","C10","C11","C12","C13","C14","C15","C16","C17","C18","C19","C20","C21","C22","C23","C24","C25","C26","C30","C31","C32","C33","C34","C37","C38","C39","C40","C41","C43","C45","C46","C47","C48","C49","C50","C51","C52","C53","C54","C55","C56","C57","C58","C60","C61","C62","C63","C64","C65","C66","C67","C68","C69","C70","C71","C72","C73","C74","C75","C76","C97"],
     "rheumatoid_arhritis":["L940","L941","L943","M05","M06","M08","M120","M123","M30","M310","M311","M312","M313","M32","M33","M34","M35","M45","M461","M468","M469"],
     "coagulopathy":["D65","D66","D67","D68","D691","D693","D694","D695","D696"],
     "obesity":["E66"],
     "weight_loss":["E40","E41","E42","E43","E44","E45","E46","R634","R64"],
     "fluid_and_electrolyte_disorders":["E222","E86","E87"],
     "blood_loss_anemia":["D500"],
     "deficiency_anemia":["D508","D509","D51","D52","D53"],
     "alcohol_abuse":["F10","E52","G621","I426","K292","K700","K703","K709","T51","Z502","Z715","Z722"],
     "drug_abuse":["F11","F12","F13","F14","F15","F16","F18","F19","Z715","Z722"],
     "psychoses":["F20","F22","F23","F24","F25","F28","F29","F302","F312","F315"],
     "depression":["F204","F313","F314","F315","F32","F33","F341","F412","F432"],
     "opioid_disorder":["F111","F112","F119"]
}

icd9_comorbidities = {
     "congestive_heart_failure": ["39891", "40211", "40291", "40411", "40413", "40491", "40493", "428"],
     "cardiac_arrhythmia":["41610", "42611", "42613", "4262", "4263", "4264", "4265", "4266", "4267", "4268", "4270", "4272", "42731", "42760", "4279", "7850", "V450", "V533"],
     "valvular_disease":["0932", "394", "395", "396", "3971", "3979", "424", "7463", "7464", "7465", "7466", "V422", "V433"],
     "pulmonary_circulation_disorder":["416","4179"],
     "peripheral vascular_disorder":["440", "4412", "4414", "4417", "4419", "4431", "4432", "4433", "4434", "4435", "4436", "4437", "4438", "4439", "4471", "5571", "5579", "V43"],
     "hypertension_uncomplicated":["4011","40119"],
     "hypertension_complicated":["40210", "40290", "40410", "40490", "4051", "4059"],
     "paralysis":["3420", "3421", "3429", "343", "344"],
     "other_neurological_disorder":["3319","3320","3334","3335","334","335","340","3411","3412","3413","3414","3415","3416","3417","3418","3419","345","3481","3483","7803","7843"],
     "chronic_pulmonary_disease":["4168","4169","490","491","492","493","494","505","5064","5081","5088"],
     "diabetes_uncomplicated":["2500", "2501", "2502", "2503"],
     "diabetes_complicated": ["2504", "2505", "2506", "2507", "2508", "2509"],
     "hypothyroidism":["243","2440","2441","2442","2448","2449"],
     "renal_failure":["40311","40391","40412","40492","585","586","V420","V451","V560","V568"],
     "liver_disease":["07032", "07033", "07054", "4560", "4561", "4562", "5710", "5712", "5713", "5714", "5715", "5716", "5717", "5718", "5719", "5723", "5728", "V427"],
     "gastric_ulcer":["53141", "53151", "531615317", "53191", "5324153251", "53261", "532753291", "53341", "5335153361", "5337", "5339153441", "53451", "534615347", "53491"],
     "aids_hiv":["042","043","044"],
     "lymphoma":["200", "201", "2020", "2021", "2022", "2023", "2025", "2026", "2027", "2028", "2029", "2030", "2038", "2386", "2733200", "201", "2020", "2021", "2022", "2023", "2025", "2026", "2027", "2028", "2029", "2030", "2038", "2386", "2733"],
     "metastatic_cancer":["196","197","198","199"],
     "solid_tumor_wo_metastasis":["140", "141", "142", "143", "144", "145", "146", "147", "148", "149", "150", "151", "152", "153", "154", "155", "156",
                                  "157", "158", "159", "160", "161", "162", "163", "164", "165", "166", "167", "168", "169", "170", "171","172","174","175",
                                 "179", "180", "181", "182", "183", "184", "185", "186", "187", "188", "189", "190", "191", "192", "193", "194", "195","V10"],
     "rheumatoid_arhritis":["7010","710","714","720","725"],
     "coagulopathy":["286", "2871", "2873", "2874", "2875"],
     "obesity":["2780"],
     "weight_loss":["260","261","262","263","7832"],
     "fluid_and_electrolyte_disorders":["276"],
     "blood_loss_anemia":["2800"],
     "deficiency_anemia":["2801", "2802", "2803", "2804", "2805", "2806", "2807", "2808", "2809", "281","2859"],
     "alcohol_abuse":["2911", "2912", "2915", "2916", "2917", "2918", "2919", "3039", "3050", "V113"],
     "drug_abuse":["2920", "29282", "29283", "29284", "29285", "29286", "29287", "29288", "29289", "2929", "3040", "3052", "3053", "3054", "3055", "3056", "3057", "3058", "30592920", "29282", "29283", "29284", "29285", "29286", "29287", "29288", "29289", "2929", "3040", "3052", "3053", "3054", "3055", "3056", "3057", "3058", "3059"],
     "psychoses":["295", "296", "297", "298", "2991"],
     "depression":["3004","30112","3090","3091","311"],
     "opioid_disorder":["30550","3040","3047"]
}

In [ ]:
dx_filter_pre_period.head()

In [ ]:
comorb_cols = list(icd10_comorbidities.keys())
dx_raw = dx_filter_pre_period.loc[:,["CLAIM_ID", "DIAG_CD","DIAG_QUALIFY_CD","CLAIM_FROM_DT","MA_NUM","OUTCOME"]]
icd10_truths = np.array(dx_filter_pre_period.DIAG_QUALIFY_CD == 10)
for comorb in comorb_cols:
    icd10_hits = np.array([any(code.startswith(num) for num in icd10_comorbidities[comorb]) for code in dx_filter_pre_period.DIAG_CD.astype(str)]) * icd10_truths
    icd9_hits = np.array([any(code.startswith(num) for num in icd9_comorbidities[comorb]) for code in dx_filter_pre_period.DIAG_CD.astype(str)]) * ~icd10_truths
    dx_raw.loc[:,comorb] = icd9_hits + icd10_hits

dx_features = dx_raw.groupby("MA_NUM").sum().drop(['DIAG_QUALIFY_CD','OUTCOME'], axis = 1 )
dx_features = dx_features.merge(dx_raw.loc[:,['OUTCOME','MA_NUM']].groupby("MA_NUM").first(),  left_index = True, right_index = True)
cancer_mas = set(dx_features.loc[np.sum(dx_features.loc[:,['lymphoma','metastatic_cancer','solid_tumor_wo_metastasis']], axis = 1) > 0].index)
print("Cancer patients identified: {}".format(len(cancer_mas)))

opioid_disorder_mas = set(dx_features.loc[np.sum(dx_features.loc[:,['opioid_disorder']], axis = 1) > 0].index)
print("Opioid disorder patients identified: {}".format(len(opioid_disorder_mas)))

dx_features.head()

In [ ]:
dx_features.to_csv(STATE + '/dx_features.csv')


cancer_mas_file = open(STATE + "/cancer_mas", "wb")
pickle.dump(cancer_mas, cancer_mas_file)
cancer_mas_file.close()


opioid_disorder_mas_file = open(STATE + "/opioid_disorder_mas", "wb")
pickle.dump(opioid_disorder_mas, opioid_disorder_mas_file)
opioid_disorder_mas_file.close()

# Eligibility Features

In [ ]:
ind_filter = pd.read_csv(STATE + "/ind_filter.csv")
ind_filter = ind_filter.loc[~ind_filter.MA_START_DT.isna()]
indc_filter = pd.read_csv(STATE + "/indc_filter.csv")
ind_filter = ind_filter.loc[([x == y for (x, y) in zip([datetime.datetime.fromisoformat(str(x)) for x in ind_filter.MA_START_DT],  [x[0][0] for x in ind_filter.MA_NUM.map(opioid_naive_info_known)])])]
                          
ind_filter.head()  

In [ ]:
#ELIGIBILITY_TIME
#Time patient was eligible for medicaid at date of prescription
ind_filter.loc[:,'ELIGIBILITY_TIME'] = np.subtract([x[0][2] for x in ind_filter.MA_NUM.map(opioid_naive_info_known)], [datetime.datetime.fromisoformat(str(x)) for x in ind_filter.MA_START_DT])
ind_filter.loc[:,'ELIGIBILITY_TIME'] = [int(x.days) for x in ind_filter.loc[:,'ELIGIBILITY_TIME']]

#elig_time_feature = eligibility_filter.groupby('MA_NUM')['AGE'].mean()
#eligibility_features = eligibility_features.merge(birth_feature.to_frame(), left_index=True, right_index = True)

elg_avg = [np.mean(ind_filter.loc[:,'ELIGIBILITY_TIME'] < i) for  i in range(100)]
plt.plot(range(100),elg_avg)
plt.title("Percent excluded by eligibility requirement")
plt.ylabel("% Excluded")
plt.xlabel("Days prior eligibility required")
plt.show()
    

In [ ]:
eligibility_filter = ind_filter.merge(indc_filter, left_on = 'INDV_ID', right_on = 'INDV_ID')

eligibility_features = eligibility_filter.loc[:,["MA_NUM", "OUTCOME"]].drop_duplicates().set_index("MA_NUM")

#AGE
#Age at date of prescription
eligibility_filter.loc[:,'AGE'] = [x[0][2].year for x in eligibility_filter.MA_NUM.map(opioid_naive_info_known)] - eligibility_filter.DOB_DT
birth_feature = eligibility_filter.groupby('MA_NUM')['AGE'].mean()
eligibility_features = eligibility_features.merge(birth_feature.to_frame(), left_index=True, right_index = True)

#ELIGIBILITY_TIME
#Time patient was eligible for medicaid at date of prescription
elig_time_feature = eligibility_filter.groupby('MA_NUM')['ELIGIBILITY_TIME'].mean()
eligibility_features = eligibility_features.merge(elig_time_feature.to_frame(), left_index=True, right_index = True)

#GENDER
#First Gender recorded in table
gender_feature = eligibility_filter.groupby('MA_NUM')['GENCD_RF'].first()
eligibility_features = eligibility_features.merge(gender_feature.to_frame(), left_index=True, right_index = True).rename(columns = {"GENCD_RF":"GENDER"})

#ZIP
zip_feature = eligibility_filter.groupby('MA_NUM')['ZIP_CD'].first()
eligibility_features = eligibility_features.merge(zip_feature.to_frame(), left_index=True, right_index = True).rename(columns = {"ZIP_CD":"ZIP"})

#city
city_feature = eligibility_filter.groupby('MA_NUM')['CITY_TXT'].first()
eligibility_features = eligibility_features.merge(city_feature.to_frame(), left_index=True, right_index = True).rename(columns = {"CITY_TXT":"CITY"})

#language
language_feature = eligibility_filter.groupby('MA_NUM')['LNGCD_RF'].first()
eligibility_features = eligibility_features.merge(language_feature.to_frame(), left_index=True, right_index = True).rename(columns = {"LNGCD_RF":"LANGUAGE"})

#ethnicity
ethnicity_feature = eligibility_filter.groupby('MA_NUM')['EGPCD_RF'].first()
eligibility_features = eligibility_features.merge(ethnicity_feature.to_frame(), left_index=True, right_index = True).rename(columns = {"EGPCD_RF":"ETHNICITY"})

eligibility_features = eligibility_features.reset_index()

youth_mas = set(eligibility_features.loc[eligibility_features.AGE < 18].index)
senior_mas = set(eligibility_features.loc[eligibility_features.AGE > 65].index)

youth_mas_file = open(STATE + "/youth_mas", "wb")
pickle.dump(youth_mas, youth_mas_file)
youth_mas_file.close()

senior_mas_file = open(STATE + "/senior_mas", "wb")
pickle.dump(senior_mas, senior_mas_file)
senior_mas_file.close()

### Going from ZIP to FIPS

In [ ]:
zipmap = pd.read_csv('Resources/zipmap.txt')
zipmap.loc[:,"FIPS"] = zipmap.STATE*1000 +zipmap.COUNTY
zipmap = zipmap.loc[:,["ZCTA5","FIPS"]].drop_duplicates(subset="ZCTA5")
eligibility_features.loc[:,'ZIP'] = pd.to_numeric(eligibility_features.ZIP.apply(lambda x: str(x)[:5]), errors='coerce')
eligibility_features = eligibility_features.merge(zipmap, how = "left", left_on = "ZIP", right_on = "ZCTA5")

unemployment_data = pd.read_csv("Resources/Unemployment.csv", header = 7)
unemployment_data.FIPStxt = unemployment_data.FIPStxt.astype(int)
eligibility_features = eligibility_features.merge(unemployment_data.loc[:,["FIPStxt","Rural_urban_continuum_code_2013","Urban_influence_code_2013","Unemployment_rate_2018", "Median_Household_Income_2018","Med_HH_Income_Percent_of_State_Total_2018"]],how = "left", left_on = "FIPS",right_on = "FIPStxt").drop("FIPStxt",axis = 1)

poverty_data = pd.read_csv("Resources/PovertyEstimates.csv", header = 4)
poverty_data.FIPStxt = poverty_data.FIPStxt.astype(int)
eligibility_features = eligibility_features.merge(poverty_data.loc[:,["FIPStxt","PCTPOVALL_2018"]],how = "left", left_on = "FIPS",right_on = "FIPStxt").drop("FIPStxt",axis = 1)

education_data = pd.read_csv("Resources/Education.csv", header = 4)
education_data.FIPStxt = education_data.FIPStxt.astype(int)
eligibility_features = eligibility_features.merge(education_data.loc[:,["FIPStxt","Percent_less_than_high_school_1418","Percent_only_high_school_1418","Percent_some_college_1418","Percent_bachelors_or_higher_1418"]],how = "left", left_on = "FIPS",right_on = "FIPStxt").drop("FIPStxt",axis = 1)

eligibility_features = eligibility_features.drop(["ZIP","ZCTA5","CITY"],axis = 1)
eligibility_features = eligibility_features.set_index("MA_NUM")

eligibility_features.head()

In [ ]:
eligibility_features.to_csv(STATE + "/eligibility_features.csv")

# Common Features

In [ ]:
common_filter = pd.read_csv(STATE + "/common_filter.csv", usecols = ['CLAIM_ID','CLAIM_TYPE_CD','SRVC_PROVIDER_ID','ORIG_SRVC_PROV_TYPE_CD','PTNT_PREG_IND','COUNTY_CD','PLACE_OF_SRVC_CD','OUTCOME','MA_NUM','CLAIM_FROM_DT'])
common_filter.CLAIM_FROM_DT = common_filter.CLAIM_FROM_DT.map(lambda x: datetime.datetime.fromisoformat(x))


#Filter for prescriptions that are before the rx date and after the 12 months before rx date
common_filter_pre_period = common_filter[([x[0][2] for x in common_filter.MA_NUM.map(opioid_naive_info_known)] > common_filter.CLAIM_FROM_DT)
                                & ([x[0][2]  - BACK_BUFFER for x in common_filter.MA_NUM.map(opioid_naive_info_known)] < common_filter.CLAIM_FROM_DT)]
common_filter_pre_period = common_filter_pre_period.loc[common_filter_pre_period.MA_NUM.isin(opioid_naive_info_known.keys())]


#Recemt - Filter for prescriptions that are before the rx date and after the 1 month before rx date
common_filter_pre_period_recent = common_filter_pre_period[([x[0][2] - BACK_BUFFER_recent for x in common_filter_pre_period.MA_NUM.map(opioid_naive_info_known)] < common_filter_pre_period.CLAIM_FROM_DT)]


del common_filter

In [ ]:
common_filter_pre_period.loc[:,'emergency_room_claim'] = common_filter_pre_period.PLACE_OF_SRVC_CD == 23
common_filter_pre_period.loc[:,'home_claim'] = common_filter_pre_period.PLACE_OF_SRVC_CD == 12
common_filter_pre_period.loc[:,'outpatient_claim'] = common_filter_pre_period.PLACE_OF_SRVC_CD == 22
common_filter_pre_period.loc[:,'inpatient_claim'] = common_filter_pre_period.PLACE_OF_SRVC_CD == 21
common_filter_pre_period.loc[:,'ambulance_claim'] = common_filter_pre_period.PLACE_OF_SRVC_CD == 41
common_filter_pre_period.loc[:,'mental_health_claim'] = common_filter_pre_period.PLACE_OF_SRVC_CD == 53
common_filter_pre_period.loc[:,'urgent_care_claim'] = common_filter_pre_period.PLACE_OF_SRVC_CD == 20
common_filter_pre_period.loc[:,'ambulatory_surgery_claim'] = common_filter_pre_period.PLACE_OF_SRVC_CD == 24

common_filter_pre_period_recent.loc[:,'emergency_room_claim'] = common_filter_pre_period_recent.PLACE_OF_SRVC_CD == 23
common_filter_pre_period_recent.loc[:,'home_claim'] = common_filter_pre_period_recent.PLACE_OF_SRVC_CD == 12
common_filter_pre_period_recent.loc[:,'outpatient_claim'] = common_filter_pre_period_recent.PLACE_OF_SRVC_CD == 22
common_filter_pre_period_recent.loc[:,'inpatient_claim'] = common_filter_pre_period_recent.PLACE_OF_SRVC_CD == 21
common_filter_pre_period_recent.loc[:,'ambulance_claim'] = common_filter_pre_period_recent.PLACE_OF_SRVC_CD == 41
common_filter_pre_period_recent.loc[:,'mental_health_claim'] = common_filter_pre_period_recent.PLACE_OF_SRVC_CD == 53
common_filter_pre_period_recent.loc[:,'urgent_care_claim'] = common_filter_pre_period_recent.PLACE_OF_SRVC_CD == 20
common_filter_pre_period_recent.loc[:,'ambulatory_surgery_claim'] = common_filter_pre_period_recent.PLACE_OF_SRVC_CD == 24


In [ ]:
pre_common_features = common_filter_pre_period.loc[:,["MA_NUM", "OUTCOME"]].drop_duplicates().set_index("MA_NUM")

num_emergency_claims = common_filter_pre_period.groupby('MA_NUM')['emergency_room_claim'].sum()
num_home_claims = common_filter_pre_period.groupby('MA_NUM')['home_claim'].sum()
num_outpatient_claims = common_filter_pre_period.groupby('MA_NUM')['outpatient_claim'].sum()
num_inpatient_claims = common_filter_pre_period.groupby('MA_NUM')['inpatient_claim'].sum()
num_ambulance_claims = common_filter_pre_period.groupby('MA_NUM')['ambulance_claim'].sum()
num_mental_health_claims = common_filter_pre_period.groupby('MA_NUM')['mental_health_claim'].sum()
num_urgent_care_claims = common_filter_pre_period.groupby('MA_NUM')['urgent_care_claim'].sum()
num_ambulatory_surgery_claims = common_filter_pre_period.groupby('MA_NUM')['ambulatory_surgery_claim'].sum()
pre_pregnancy_status = common_filter_pre_period.groupby('MA_NUM')['PTNT_PREG_IND'].sum()
pre_total_claims = common_filter_pre_period.groupby('MA_NUM')['CLAIM_ID'].nunique()

pre_common_features = pre_common_features.merge(num_emergency_claims.to_frame(), left_index=True, right_index = True)
pre_common_features = pre_common_features.merge(num_home_claims.to_frame(), left_index=True, right_index = True)
pre_common_features = pre_common_features.merge(num_outpatient_claims.to_frame(), left_index=True, right_index = True)
pre_common_features = pre_common_features.merge(num_inpatient_claims.to_frame(), left_index=True, right_index = True)
pre_common_features = pre_common_features.merge(num_ambulance_claims.to_frame(), left_index=True, right_index = True)
pre_common_features = pre_common_features.merge(num_mental_health_claims.to_frame(), left_index=True, right_index = True)
pre_common_features = pre_common_features.merge(num_urgent_care_claims.to_frame(), left_index=True, right_index = True)
pre_common_features = pre_common_features.merge(num_ambulatory_surgery_claims.to_frame(), left_index=True, right_index = True)
pre_common_features = pre_common_features.merge(pre_pregnancy_status.to_frame(), left_index=True, right_index = True).rename(columns = {"PTNT_PREG_IND":"PREVIOUSLY_PREGNANT"})
pre_common_features = pre_common_features.merge(pre_total_claims.to_frame(), left_index=True, right_index = True).rename(columns = {"CLAIM_ID":"TOTAL_PRE_CLAIMS"})
pre_common_features.head()

In [ ]:
recent_common_features = common_filter_pre_period_recent.loc[:,["MA_NUM", "OUTCOME"]].drop_duplicates().set_index("MA_NUM")

num_emergency_claims = common_filter_pre_period_recent.groupby('MA_NUM')['emergency_room_claim'].sum()
num_home_claims = common_filter_pre_period_recent.groupby('MA_NUM')['home_claim'].sum()
num_outpatient_claims = common_filter_pre_period_recent.groupby('MA_NUM')['outpatient_claim'].sum()
num_inpatient_claims = common_filter_pre_period_recent.groupby('MA_NUM')['inpatient_claim'].sum()
num_ambulance_claims = common_filter_pre_period_recent.groupby('MA_NUM')['ambulance_claim'].sum()
num_mental_health_claims = common_filter_pre_period_recent.groupby('MA_NUM')['mental_health_claim'].sum()
num_urgent_care_claims = common_filter_pre_period_recent.groupby('MA_NUM')['urgent_care_claim'].sum()
num_ambulatory_surgery_claims = common_filter_pre_period_recent.groupby('MA_NUM')['ambulatory_surgery_claim'].sum()
pre_pregnancy_status = common_filter_pre_period_recent.groupby('MA_NUM')['PTNT_PREG_IND'].sum()
pre_total_claims = common_filter_pre_period_recent.groupby('MA_NUM')['CLAIM_ID'].nunique()

recent_common_features = recent_common_features.merge(num_emergency_claims.to_frame(), left_index=True, right_index = True)
recent_common_features = recent_common_features.merge(num_home_claims.to_frame(), left_index=True, right_index = True)
recent_common_features = recent_common_features.merge(num_outpatient_claims.to_frame(), left_index=True, right_index = True)
recent_common_features = recent_common_features.merge(num_inpatient_claims.to_frame(), left_index=True, right_index = True)
recent_common_features = recent_common_features.merge(num_ambulance_claims.to_frame(), left_index=True, right_index = True)
recent_common_features = recent_common_features.merge(num_mental_health_claims.to_frame(), left_index=True, right_index = True)
recent_common_features = recent_common_features.merge(num_urgent_care_claims.to_frame(), left_index=True, right_index = True)
recent_common_features = recent_common_features.merge(num_ambulatory_surgery_claims.to_frame(), left_index=True, right_index = True)
recent_common_features = recent_common_features.merge(pre_pregnancy_status.to_frame(), left_index=True, right_index = True).rename(columns = {"PTNT_PREG_IND":"PREVIOUSLY_PREGNANT"})
recent_common_features = recent_common_features.merge(pre_total_claims.to_frame(), left_index=True, right_index = True).rename(columns = {"CLAIM_ID":"TOTAL_PRE_CLAIMS"})
recent_common_features = recent_common_features.add_suffix('_recent').rename(columns = {"OUTCOME_recent":"OUTCOME"})
recent_common_features.head()

In [ ]:
pre_common_features.to_csv(STATE + "/pre_common_features.csv")
recent_common_features.to_csv(STATE + "/recent_common_features.csv")

# Pre RX features

In [ ]:
rx_filter = pd.read_csv(STATE + "/rx_filter.csv")
rx_filter = rx_filter.loc[rx_filter.MA_PAID_AMT > 0]
rx_filter = rx_filter.loc[rx_filter.DISPENSE_QTY_AMT > 0]
rx_filter.RX_WRITTEN_DT = rx_filter.RX_WRITTEN_DT.map(lambda x: datetime.datetime.fromisoformat(x))
    
#Filter for prescriptions that are before the rx date and after the 12 months before rx date
rx_filter_pre_period = rx_filter[([x[0][2] for x in rx_filter.MA_NUM.map(opioid_naive_info_known)] > rx_filter.RX_WRITTEN_DT)
                                & ([x[0][2]  - BACK_BUFFER for x in rx_filter.MA_NUM.map(opioid_naive_info_known)] < rx_filter.RX_WRITTEN_DT)]
rx_filter_pre_period = rx_filter_pre_period.loc[rx_filter_pre_period.MA_NUM.isin(opioid_naive_info_known.keys())]

#Recent - Filter for prescriptions that are before the rx date and after the 1 month1 before rx date
rx_filter_pre_period_recent = rx_filter_pre_period[([x[0][2]  - BACK_BUFFER_recent for x in rx_filter_pre_period.MA_NUM.map(opioid_naive_info_known)] < rx_filter_pre_period.RX_WRITTEN_DT)]


del rx_filter

In [ ]:
rx_filter_pre_period.loc[:,'naltrexone_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(naltrexone_ndcs)
rx_filter_pre_period.loc[:,'fluoxetine_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(fluoxetine_ndcs)
rx_filter_pre_period.loc[:,'buprenorphine_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(buprenorphine_ndcs)
rx_filter_pre_period.loc[:,'bupropion_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(bupropion_ndcs)
rx_filter_pre_period.loc[:,'dacomitinib_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(dacomitinib_ndcs)
rx_filter_pre_period.loc[:,'paroxetine_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(paroxetine_ndcs)
rx_filter_pre_period.loc[:,'quinidine_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(quinidine_ndcs)

rx_filter_pre_period.loc[:,'acetaminophen_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(acetaminophen_ndcs)
rx_filter_pre_period.loc[:,'ibuprofen_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(ibuprofen_ndcs)
rx_filter_pre_period.loc[:,'diclofenac_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(diclofenac_ndcs)
rx_filter_pre_period.loc[:,'ketorolac_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(ketorolac_ndcs)
rx_filter_pre_period.loc[:,'naproxen_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(naproxen_ndcs)

rx_filter_pre_period.loc[:,'duloxetine_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(duloxetine_ndcs)
rx_filter_pre_period.loc[:,'milnacipran_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(milnacipran_ndcs)
rx_filter_pre_period.loc[:,'venlafaxin_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(venlafaxin_ndcs)
rx_filter_pre_period.loc[:,'diclofenac_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(diclofenac_ndcs)
rx_filter_pre_period.loc[:,'gabapentin_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(gabapentin_ndcs)
rx_filter_pre_period.loc[:,'pregabalin_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(pregabalin_ndcs)
rx_filter_pre_period.loc[:,'amitriptalin_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(amitriptalin_ndcs)
rx_filter_pre_period.loc[:,'doxipen_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(doxipen_ndcs)
rx_filter_pre_period.loc[:,'imipramine_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(imipramine_ndcs)
rx_filter_pre_period.loc[:,'desipramine_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(desipramine_ndcs)
rx_filter_pre_period.loc[:,'nortryptaline_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(nortryptaline_ndcs)
rx_filter_pre_period.loc[:,'lidocaine_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(lidocaine_ndcs)
rx_filter_pre_period.loc[:,'capsaicin_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(capsaicin_ndcs)



pre_rx_features = rx_filter_pre_period.loc[:,["MA_NUM", "OUTCOME"]].drop_duplicates().set_index("MA_NUM")

#Creates general features about the rx's given to the patient 12 months prior
num_unique_ndcs = rx_filter_pre_period.groupby('MA_NUM')['PRODUCT_CD'].nunique()
tot_ndcs = rx_filter_pre_period.groupby('MA_NUM')['PRODUCT_CD'].count()
total_rx_billed = rx_filter_pre_period.groupby('MA_NUM')['MA_BILLED_AMT'].sum()
total_rx_paid = rx_filter_pre_period.groupby('MA_NUM')['MA_PAID_AMT'].sum()
avg_rx_billed = rx_filter_pre_period.groupby('MA_NUM')['MA_BILLED_AMT'].mean()
avg_rx_paid = rx_filter_pre_period.groupby('MA_NUM')['MA_PAID_AMT'].mean()
avg_rx_dispense = rx_filter_pre_period.groupby('MA_NUM')['DISPENSE_QTY_AMT'].mean()
avg_rx_days = rx_filter_pre_period.groupby('MA_NUM')['DAYS_SUPPLY_NUM'].mean()
total_rx_dispense = rx_filter_pre_period.groupby('MA_NUM')['DISPENSE_QTY_AMT'].sum()
total_rx_days = rx_filter_pre_period.groupby('MA_NUM')['DAYS_SUPPLY_NUM'].sum()

naltrexone_ndcs = rx_filter_pre_period.groupby("MA_NUM")['naltrexone_ndcs'].sum()
fluoxetine_ndcs = rx_filter_pre_period.groupby("MA_NUM")['fluoxetine_ndcs'].sum()
buprenorphine_ndcs = rx_filter_pre_period.groupby("MA_NUM")['buprenorphine_ndcs'].sum()
bupropion_ndcs = rx_filter_pre_period.groupby("MA_NUM")['bupropion_ndcs'].sum()
dacomitinib_ndcs = rx_filter_pre_period.groupby("MA_NUM")['dacomitinib_ndcs'].sum()
paroxetine_ndcs = rx_filter_pre_period.groupby("MA_NUM")['paroxetine_ndcs'].sum()
quinidine_ndcs = rx_filter_pre_period.groupby("MA_NUM")['quinidine_ndcs'].sum()

acetaminophen_ndcs = rx_filter_pre_period.groupby("MA_NUM")['acetaminophen_ndcs'].sum()
ibuprofen_ndcs = rx_filter_pre_period.groupby("MA_NUM")['ibuprofen_ndcs'].sum()
diclofenac_ndcs = rx_filter_pre_period.groupby("MA_NUM")['diclofenac_ndcs'].sum()
ketorolac_ndcs = rx_filter_pre_period.groupby("MA_NUM")['ketorolac_ndcs'].sum()
naproxen_ndcs = rx_filter_pre_period.groupby("MA_NUM")['naproxen_ndcs'].sum()


duloxetine_ndcs = rx_filter_pre_period.groupby("MA_NUM")['duloxetine_ndcs'].sum()
milnacipran_ndcs = rx_filter_pre_period.groupby("MA_NUM")['milnacipran_ndcs'].sum()
venlafaxin_ndcs = rx_filter_pre_period.groupby("MA_NUM")['venlafaxin_ndcs'].sum()
diclofenac_ndcs = rx_filter_pre_period.groupby("MA_NUM")['diclofenac_ndcs'].sum()
gabapentin_ndcs = rx_filter_pre_period.groupby("MA_NUM")['gabapentin_ndcs'].sum()
pregabalin_ndcs = rx_filter_pre_period.groupby("MA_NUM")['pregabalin_ndcs'].sum()
amitriptalin_ndcs = rx_filter_pre_period.groupby("MA_NUM")['amitriptalin_ndcs'].sum()
doxipen_ndcs = rx_filter_pre_period.groupby("MA_NUM")['doxipen_ndcs'].sum()
imipramine_ndcs = rx_filter_pre_period.groupby("MA_NUM")['imipramine_ndcs'].sum()
desipramine_ndcs = rx_filter_pre_period.groupby("MA_NUM")['desipramine_ndcs'].sum()
nortryptaline_ndcs = rx_filter_pre_period.groupby("MA_NUM")['nortryptaline_ndcs'].sum()
lidocaine_ndcs = rx_filter_pre_period.groupby("MA_NUM")['lidocaine_ndcs'].sum()
capsaicin_ndcs = rx_filter_pre_period.groupby("MA_NUM")['capsaicin_ndcs'].sum()


#Adds all these features to the feature matrix

pre_rx_features = pre_rx_features.merge(num_unique_ndcs.to_frame(), left_index=True, right_index = True).rename(columns = {"PRODUCT_CD":"PRE_NUM_POLYPHARMACY"})
pre_rx_features = pre_rx_features.merge(tot_ndcs.to_frame(), left_index=True, right_index = True).rename(columns = {"PRODUCT_CD":"TOTAL_RXS"})
pre_rx_features = pre_rx_features.merge(total_rx_billed.to_frame(), left_index=True, right_index = True).rename(columns = {"MA_BILLED_AMT":"PRE_RX_BILLED_SUM"})
pre_rx_features = pre_rx_features.merge(total_rx_paid.to_frame(), left_index=True, right_index = True).rename(columns = {"MA_PAID_AMT":"PRE_RX_PAID_SUM"})
pre_rx_features = pre_rx_features.merge(avg_rx_billed.to_frame(), left_index=True, right_index = True).rename(columns = {"MA_BILLED_AMT":"PRE_RX_BILLED_AVG"})
pre_rx_features = pre_rx_features.merge(avg_rx_paid.to_frame(), left_index=True, right_index = True).rename(columns = {"MA_PAID_AMT":"PRE_RX_PAID_AVG"})
pre_rx_features = pre_rx_features.merge(avg_rx_dispense.to_frame(), left_index=True, right_index = True).rename(columns = {"DISPENSE_QTY_AMT":"PRE_RX_QTY_AVG"})
pre_rx_features = pre_rx_features.merge(avg_rx_days.to_frame(), left_index=True, right_index = True).rename(columns = {"DAYS_SUPPLY_NUM":"PRE_RX_DAYS_AVD"})
pre_rx_features = pre_rx_features.merge(total_rx_dispense.to_frame(), left_index=True, right_index = True).rename(columns = {"DISPENSE_QTY_AMT":"PRE_RX_QTY_SUM"})
pre_rx_features = pre_rx_features.merge(total_rx_days.to_frame(), left_index=True, right_index = True).rename(columns = {"DAYS_SUPPLY_NUM":"PRE_RX_DAYS_SUM"})

pre_rx_features = pre_rx_features.merge(naltrexone_ndcs.to_frame(), left_index=True, right_index = True)
pre_rx_features = pre_rx_features.merge(fluoxetine_ndcs.to_frame(), left_index=True, right_index = True)
pre_rx_features = pre_rx_features.merge(buprenorphine_ndcs.to_frame(), left_index=True, right_index = True)
pre_rx_features = pre_rx_features.merge(bupropion_ndcs.to_frame(), left_index=True, right_index = True)
pre_rx_features = pre_rx_features.merge(dacomitinib_ndcs.to_frame(), left_index=True, right_index = True)
pre_rx_features = pre_rx_features.merge(paroxetine_ndcs.to_frame(), left_index=True, right_index = True)
pre_rx_features = pre_rx_features.merge(quinidine_ndcs.to_frame(), left_index=True, right_index = True)

pre_rx_features = pre_rx_features.merge(acetaminophen_ndcs.to_frame(), left_index=True, right_index = True)
pre_rx_features = pre_rx_features.merge(ibuprofen_ndcs.to_frame(), left_index=True, right_index = True)
pre_rx_features = pre_rx_features.merge(diclofenac_ndcs.to_frame(), left_index=True, right_index = True)
pre_rx_features = pre_rx_features.merge(ketorolac_ndcs.to_frame(), left_index=True, right_index = True)
pre_rx_features = pre_rx_features.merge(naproxen_ndcs.to_frame(), left_index=True, right_index = True)


pre_rx_features = pre_rx_features.merge(duloxetine_ndcs.to_frame(), left_index=True, right_index = True)
pre_rx_features = pre_rx_features.merge(milnacipran_ndcs.to_frame(), left_index=True, right_index = True)
pre_rx_features = pre_rx_features.merge(venlafaxin_ndcs.to_frame(), left_index=True, right_index = True)
pre_rx_features = pre_rx_features.merge(diclofenac_ndcs.to_frame(), left_index=True, right_index = True)
pre_rx_features = pre_rx_features.merge(gabapentin_ndcs.to_frame(), left_index=True, right_index = True)
pre_rx_features = pre_rx_features.merge(pregabalin_ndcs.to_frame(), left_index=True, right_index = True)
pre_rx_features = pre_rx_features.merge(amitriptalin_ndcs.to_frame(), left_index=True, right_index = True)
pre_rx_features = pre_rx_features.merge(doxipen_ndcs.to_frame(), left_index=True, right_index = True)
pre_rx_features = pre_rx_features.merge(imipramine_ndcs.to_frame(), left_index=True, right_index = True)
pre_rx_features = pre_rx_features.merge(desipramine_ndcs.to_frame(), left_index=True, right_index = True)
pre_rx_features = pre_rx_features.merge(nortryptaline_ndcs.to_frame(), left_index=True, right_index = True)
pre_rx_features = pre_rx_features.merge(lidocaine_ndcs.to_frame(), left_index=True, right_index = True)
pre_rx_features = pre_rx_features.merge(capsaicin_ndcs.to_frame(), left_index=True, right_index = True)

pre_rx_features.head()

In [ ]:
rx_filter_pre_period_recent.loc[:,'naltrexone_ndcs'] = rx_filter_pre_period_recent.PRODUCT_CD.isin(naltrexone_ndcs)
rx_filter_pre_period_recent.loc[:,'fluoxetine_ndcs'] = rx_filter_pre_period_recent.PRODUCT_CD.isin(fluoxetine_ndcs)
rx_filter_pre_period_recent.loc[:,'buprenorphine_ndcs'] = rx_filter_pre_period_recent.PRODUCT_CD.isin(buprenorphine_ndcs)
rx_filter_pre_period_recent.loc[:,'bupropion_ndcs'] = rx_filter_pre_period_recent.PRODUCT_CD.isin(bupropion_ndcs)
rx_filter_pre_period_recent.loc[:,'dacomitinib_ndcs'] = rx_filter_pre_period_recent.PRODUCT_CD.isin(dacomitinib_ndcs)
rx_filter_pre_period_recent.loc[:,'paroxetine_ndcs'] = rx_filter_pre_period_recent.PRODUCT_CD.isin(paroxetine_ndcs)
rx_filter_pre_period_recent.loc[:,'quinidine_ndcs'] = rx_filter_pre_period_recent.PRODUCT_CD.isin(quinidine_ndcs)

rx_filter_pre_period_recent.loc[:,'acetaminophen_ndcs'] = rx_filter_pre_period_recent.PRODUCT_CD.isin(acetaminophen_ndcs)
rx_filter_pre_period_recent.loc[:,'ibuprofen_ndcs'] = rx_filter_pre_period_recent.PRODUCT_CD.isin(ibuprofen_ndcs)
rx_filter_pre_period_recent.loc[:,'diclofenac_ndcs'] = rx_filter_pre_period_recent.PRODUCT_CD.isin(diclofenac_ndcs)
rx_filter_pre_period_recent.loc[:,'ketorolac_ndcs'] = rx_filter_pre_period_recent.PRODUCT_CD.isin(ketorolac_ndcs)
rx_filter_pre_period_recent.loc[:,'naproxen_ndcs'] = rx_filter_pre_period_recent.PRODUCT_CD.isin(naproxen_ndcs)

rx_filter_pre_period_recent.loc[:,'duloxetine_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(duloxetine_ndcs)
rx_filter_pre_period_recent.loc[:,'milnacipran_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(milnacipran_ndcs)
rx_filter_pre_period_recent.loc[:,'venlafaxin_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(venlafaxin_ndcs)
rx_filter_pre_period_recent.loc[:,'diclofenac_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(diclofenac_ndcs)
rx_filter_pre_period_recent.loc[:,'gabapentin_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(gabapentin_ndcs)
rx_filter_pre_period_recent.loc[:,'pregabalin_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(pregabalin_ndcs)
rx_filter_pre_period_recent.loc[:,'amitriptalin_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(amitriptalin_ndcs)
rx_filter_pre_period_recent.loc[:,'doxipen_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(doxipen_ndcs)
rx_filter_pre_period_recent.loc[:,'imipramine_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(imipramine_ndcs)
rx_filter_pre_period_recent.loc[:,'desipramine_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(desipramine_ndcs)
rx_filter_pre_period_recent.loc[:,'nortryptaline_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(nortryptaline_ndcs)
rx_filter_pre_period_recent.loc[:,'lidocaine_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(lidocaine_ndcs)
rx_filter_pre_period_recent.loc[:,'capsaicin_ndcs'] = rx_filter_pre_period.PRODUCT_CD.isin(capsaicin_ndcs)

recent_rx_features = rx_filter_pre_period_recent.loc[:,["MA_NUM", "OUTCOME"]].drop_duplicates().set_index("MA_NUM")

#Creates general features about the rx's given to the patient 12 months prior
num_unique_ndcs = rx_filter_pre_period_recent.groupby('MA_NUM')['PRODUCT_CD'].nunique()
tot_ndcs = rx_filter_pre_period_recent.groupby('MA_NUM')['PRODUCT_CD'].count()
total_rx_billed = rx_filter_pre_period_recent.groupby('MA_NUM')['MA_BILLED_AMT'].sum()
total_rx_paid = rx_filter_pre_period_recent.groupby('MA_NUM')['MA_PAID_AMT'].sum()
avg_rx_billed = rx_filter_pre_period_recent.groupby('MA_NUM')['MA_BILLED_AMT'].mean()
avg_rx_paid = rx_filter_pre_period_recent.groupby('MA_NUM')['MA_PAID_AMT'].mean()
avg_rx_dispense = rx_filter_pre_period_recent.groupby('MA_NUM')['DISPENSE_QTY_AMT'].mean()
avg_rx_days = rx_filter_pre_period_recent.groupby('MA_NUM')['DAYS_SUPPLY_NUM'].mean()
total_rx_dispense = rx_filter_pre_period_recent.groupby('MA_NUM')['DISPENSE_QTY_AMT'].sum()
total_rx_days = rx_filter_pre_period_recent.groupby('MA_NUM')['DAYS_SUPPLY_NUM'].sum()

naltrexone_ndcs = rx_filter_pre_period_recent.groupby("MA_NUM")['naltrexone_ndcs'].sum()
fluoxetine_ndcs = rx_filter_pre_period_recent.groupby("MA_NUM")['fluoxetine_ndcs'].sum()
buprenorphine_ndcs = rx_filter_pre_period_recent.groupby("MA_NUM")['buprenorphine_ndcs'].sum()
bupropion_ndcs = rx_filter_pre_period_recent.groupby("MA_NUM")['bupropion_ndcs'].sum()
dacomitinib_ndcs = rx_filter_pre_period_recent.groupby("MA_NUM")['dacomitinib_ndcs'].sum()
paroxetine_ndcs = rx_filter_pre_period_recent.groupby("MA_NUM")['paroxetine_ndcs'].sum()
quinidine_ndcs = rx_filter_pre_period_recent.groupby("MA_NUM")['quinidine_ndcs'].sum()

acetaminophen_ndcs = rx_filter_pre_period_recent.groupby("MA_NUM")['acetaminophen_ndcs'].sum()
ibuprofen_ndcs = rx_filter_pre_period_recent.groupby("MA_NUM")['ibuprofen_ndcs'].sum()
diclofenac_ndcs = rx_filter_pre_period_recent.groupby("MA_NUM")['diclofenac_ndcs'].sum()
ketorolac_ndcs = rx_filter_pre_period_recent.groupby("MA_NUM")['ketorolac_ndcs'].sum()
naproxen_ndcs = rx_filter_pre_period_recent.groupby("MA_NUM")['naproxen_ndcs'].sum()

duloxetine_ndcs = rx_filter_pre_period_recent.groupby("MA_NUM")['duloxetine_ndcs'].sum()
milnacipran_ndcs = rx_filter_pre_period_recent.groupby("MA_NUM")['milnacipran_ndcs'].sum()
venlafaxin_ndcs = rx_filter_pre_period_recent.groupby("MA_NUM")['venlafaxin_ndcs'].sum()
diclofenac_ndcs = rx_filter_pre_period_recent.groupby("MA_NUM")['diclofenac_ndcs'].sum()
gabapentin_ndcs = rx_filter_pre_period_recent.groupby("MA_NUM")['gabapentin_ndcs'].sum()
pregabalin_ndcs = rx_filter_pre_period_recent.groupby("MA_NUM")['pregabalin_ndcs'].sum()
amitriptalin_ndcs = rx_filter_pre_period_recent.groupby("MA_NUM")['amitriptalin_ndcs'].sum()
doxipen_ndcs = rx_filter_pre_period_recent.groupby("MA_NUM")['doxipen_ndcs'].sum()
imipramine_ndcs = rx_filter_pre_period_recent.groupby("MA_NUM")['imipramine_ndcs'].sum()
desipramine_ndcs = rx_filter_pre_period_recent.groupby("MA_NUM")['desipramine_ndcs'].sum()
nortryptaline_ndcs = rx_filter_pre_period_recent.groupby("MA_NUM")['nortryptaline_ndcs'].sum()
lidocaine_ndcs = rx_filter_pre_period_recent.groupby("MA_NUM")['lidocaine_ndcs'].sum()
capsaicin_ndcs = rx_filter_pre_period_recent.groupby("MA_NUM")['capsaicin_ndcs'].sum()


#Adds all these features to the feature matrix

recent_rx_features = recent_rx_features.merge(num_unique_ndcs.to_frame(), left_index=True, right_index = True).rename(columns = {"PRODUCT_CD":"PRE_NUM_POLYPHARMACY"})
recent_rx_features = recent_rx_features.merge(tot_ndcs.to_frame(), left_index=True, right_index = True).rename(columns = {"PRODUCT_CD":"TOTAL_RXS"})
recent_rx_features = recent_rx_features.merge(total_rx_billed.to_frame(), left_index=True, right_index = True).rename(columns = {"MA_BILLED_AMT":"PRE_RX_BILLED_SUM"})
recent_rx_features = recent_rx_features.merge(total_rx_paid.to_frame(), left_index=True, right_index = True).rename(columns = {"MA_PAID_AMT":"PRE_RX_PAID_SUM"})
recent_rx_features = recent_rx_features.merge(avg_rx_billed.to_frame(), left_index=True, right_index = True).rename(columns = {"MA_BILLED_AMT":"PRE_RX_BILLED_AVG"})
recent_rx_features = recent_rx_features.merge(avg_rx_paid.to_frame(), left_index=True, right_index = True).rename(columns = {"MA_PAID_AMT":"PRE_RX_PAID_AVG"})
recent_rx_features = recent_rx_features.merge(avg_rx_dispense.to_frame(), left_index=True, right_index = True).rename(columns = {"DISPENSE_QTY_AMT":"PRE_RX_QTY_AVG"})
recent_rx_features = recent_rx_features.merge(avg_rx_days.to_frame(), left_index=True, right_index = True).rename(columns = {"DAYS_SUPPLY_NUM":"PRE_RX_DAYS_AVD"})
recent_rx_features = recent_rx_features.merge(total_rx_dispense.to_frame(), left_index=True, right_index = True).rename(columns = {"DISPENSE_QTY_AMT":"PRE_RX_QTY_SUM"})
recent_rx_features = recent_rx_features.merge(total_rx_days.to_frame(), left_index=True, right_index = True).rename(columns = {"DAYS_SUPPLY_NUM":"PRE_RX_DAYS_SUM"})

recent_rx_features = recent_rx_features.merge(naltrexone_ndcs.to_frame(), left_index=True, right_index = True)
recent_rx_features = recent_rx_features.merge(fluoxetine_ndcs.to_frame(), left_index=True, right_index = True)
recent_rx_features = recent_rx_features.merge(buprenorphine_ndcs.to_frame(), left_index=True, right_index = True)
recent_rx_features = recent_rx_features.merge(bupropion_ndcs.to_frame(), left_index=True, right_index = True)
recent_rx_features = recent_rx_features.merge(dacomitinib_ndcs.to_frame(), left_index=True, right_index = True)
recent_rx_features = recent_rx_features.merge(paroxetine_ndcs.to_frame(), left_index=True, right_index = True)
recent_rx_features = recent_rx_features.merge(quinidine_ndcs.to_frame(), left_index=True, right_index = True)

recent_rx_features = recent_rx_features.merge(acetaminophen_ndcs.to_frame(), left_index=True, right_index = True)
recent_rx_features = recent_rx_features.merge(ibuprofen_ndcs.to_frame(), left_index=True, right_index = True)
recent_rx_features = recent_rx_features.merge(diclofenac_ndcs.to_frame(), left_index=True, right_index = True)
recent_rx_features = recent_rx_features.merge(ketorolac_ndcs.to_frame(), left_index=True, right_index = True)
recent_rx_features = recent_rx_features.merge(naproxen_ndcs.to_frame(), left_index=True, right_index = True)

recent_rx_features = recent_rx_features.merge(duloxetine_ndcs.to_frame(), left_index=True, right_index = True)
recent_rx_features = recent_rx_features.merge(milnacipran_ndcs.to_frame(), left_index=True, right_index = True)
recent_rx_features = recent_rx_features.merge(venlafaxin_ndcs.to_frame(), left_index=True, right_index = True)
recent_rx_features = recent_rx_features.merge(diclofenac_ndcs.to_frame(), left_index=True, right_index = True)
recent_rx_features = recent_rx_features.merge(gabapentin_ndcs.to_frame(), left_index=True, right_index = True)
recent_rx_features = recent_rx_features.merge(pregabalin_ndcs.to_frame(), left_index=True, right_index = True)
recent_rx_features = recent_rx_features.merge(amitriptalin_ndcs.to_frame(), left_index=True, right_index = True)
recent_rx_features = recent_rx_features.merge(doxipen_ndcs.to_frame(), left_index=True, right_index = True)
recent_rx_features = recent_rx_features.merge(imipramine_ndcs.to_frame(), left_index=True, right_index = True)
recent_rx_features = recent_rx_features.merge(desipramine_ndcs.to_frame(), left_index=True, right_index = True)
recent_rx_features = recent_rx_features.merge(nortryptaline_ndcs.to_frame(), left_index=True, right_index = True)
recent_rx_features = recent_rx_features.merge(lidocaine_ndcs.to_frame(), left_index=True, right_index = True)
recent_rx_features = recent_rx_features.merge(capsaicin_ndcs.to_frame(), left_index=True, right_index = True)

recent_rx_features = recent_rx_features.add_suffix('_recent').rename(columns = {"OUTCOME_recent":"OUTCOME"})


recent_rx_features.head()

In [ ]:
pre_rx_features.to_csv(STATE + "/pre_rx_features.csv")
recent_rx_features.to_csv(STATE + "/recent_rx_features.csv")

# Initial opioid RX features

In [ ]:
rx_filter = pd.read_csv(STATE + "/rx_filter.csv")
eligibility_features = pd.read_csv(STATE + "/eligibility_features.csv")

try:
    cancer_mas
except NameError:
    pickle_in = open(STATE + "/cancer_mas","rb")
    cancer_mas = pickle.load(pickle_in)
    
try:
    opioid_disorder_mas
except NameError:
    pickle_in = open(STATE + "/opioid_disorder_mas","rb")
    opioid_disorder_mas = pickle.load(pickle_in)

try:
    youth_mas
except NameError:
    pickle_in = open(STATE + "/youth_mas","rb")
    youth_mas = pickle.load(pickle_in)
    
try:
    senior_mas
except NameError:
    pickle_in = open(STATE + "/senior_mas","rb")
    senior_mas = pickle.load(pickle_in)
    
common_denied = pd.read_csv(STATE+ "/common_filter.csv", usecols = ["CLAIM_ID", "TRANS_STATUS_CD"])
common_denied_claims = common_denied.loc[common_denied.TRANS_STATUS_CD == 'D'].CLAIM_ID
del common_denied

In [ ]:
initial_opioid_rxs = rx_filter.loc[rx_filter.CLAIM_ID.isin([opioid_naive_info_known[ma][0][3] for ma in opioid_naive_info_known])]

In [ ]:
initial_opioid_rxs.loc[:,'PRODUCT_CD'] = initial_opioid_rxs.PRODUCT_CD.astype(float)

In [ ]:
initial_opioid_rxs.head()

In [ ]:
len(initial_opioid_rxs)

In [ ]:
initial_opioid_rxs = initial_opioid_rxs.loc[initial_opioid_rxs.MA_NUM.isin(eligibility_features.loc[eligibility_features.ELIGIBILITY_TIME > 2*30].MA_NUM)]
len(initial_opioid_rxs)

In [ ]:
#Filter out anything that's not the first fill of opioids
initial_opioid_rxs = initial_opioid_rxs.loc[initial_opioid_rxs.FILL_NUM == 0]

In [ ]:
len(initial_opioid_rxs)

In [ ]:
#Note: there are duplicate claim_ids, 
#since the length of this is longer than the medicaid id's identified
initial_opioid_rxs = initial_opioid_rxs.drop_duplicates(subset = 'MA_NUM')

In [ ]:
len(initial_opioid_rxs)

In [ ]:
#Filter out negative dispense quantity amounts
##NOTE: THIS IS A LARGE AMOUNT OF FILTERING
initial_opioid_rxs = initial_opioid_rxs.loc[initial_opioid_rxs.DISPENSE_QTY_AMT > 0]
len(initial_opioid_rxs)

In [ ]:
#Filter out denied claims, identified by MA_PAID_AMT = 0
#HALF OF THE PRESCRIPTIONS ARE DENIED...?
initial_opioid_rxs = initial_opioid_rxs.loc[~initial_opioid_rxs.CLAIM_ID.isin(common_denied_claims)]
len(initial_opioid_rxs)

In [ ]:
#Filter out cancer patients
initial_opioid_rxs = initial_opioid_rxs.loc[~initial_opioid_rxs.MA_NUM.isin(cancer_mas)]
len(initial_opioid_rxs)

In [ ]:
#Filter out opioid diagnosis patients
initial_opioid_rxs = initial_opioid_rxs.loc[~initial_opioid_rxs.MA_NUM.isin(opioid_disorder_mas)]
len(initial_opioid_rxs)

In [ ]:
#Filter out patients <18
initial_opioid_rxs = initial_opioid_rxs.loc[initial_opioid_rxs.MA_NUM.isin(eligibility_features.loc[eligibility_features.AGE >= 18].MA_NUM)]
len(initial_opioid_rxs)

In [ ]:
#Filter out patients >65
initial_opioid_rxs = initial_opioid_rxs.loc[initial_opioid_rxs.MA_NUM.isin(eligibility_features.loc[eligibility_features.AGE <= 65].MA_NUM)]
len(initial_opioid_rxs)

In [ ]:
initial_opioid_rxs = initial_opioid_rxs.merge(opioid_info, how = "left", left_on = 'PRODUCT_CD', right_on = 'NDC_Numeric')

In [ ]:
initial_opioid_rxs.loc[:,'MED'] =  initial_opioid_rxs.loc[:,'Strength_Per_Unit']*(initial_opioid_rxs.loc[:,'DISPENSE_QTY_AMT']/initial_opioid_rxs.loc[:,'DAYS_SUPPLY_NUM'])*initial_opioid_rxs.loc[:,'MME_Conversion_Factor']


In [ ]:
Counter(initial_opioid_rxs.Drug)

In [ ]:
#Filter out buprenorphine prescriptions
initial_opioid_rxs = initial_opioid_rxs.loc[initial_opioid_rxs.Drug != 'Buprenorphine']
len(initial_opioid_rxs)

In [ ]:
initial_opioid_rxs.head()

In [ ]:
outcomes = initial_opioid_rxs.groupby(['OUTCOME']).count().CLAIM_ID
outcomes

In [ ]:
drugs_by_outcome = initial_opioid_rxs.groupby(['OUTCOME','Drug'])['CLAIM_ID'].count().reset_index().pivot(index='OUTCOME', columns='Drug', values='CLAIM_ID')
drugs_by_outcome.loc['acute_freq'] = drugs_by_outcome.loc[0]/np.sum(drugs_by_outcome.loc[0])
drugs_by_outcome.loc['chronic_freq'] = drugs_by_outcome.loc[1]/np.sum(drugs_by_outcome.loc[1])
drugs_by_outcome

### Merge providers

In [ ]:
common_filter = pd.read_csv(STATE + "/common_filter.csv", usecols = ['MA_NUM','CLAIM_ID','CLAIM_TYPE_CD','SRVC_PROVIDER_ID','PTNT_PREG_IND','PLACE_OF_SRVC_CD','CLAIM_FROM_DT'])


In [ ]:
common_filter.head()

In [ ]:
Counter(common_filter.head(10000000).CLAIM_TYPE_CD)

In [ ]:
common_filter.loc[:,"DateMACombo"] = common_filter.MA_NUM.str.cat(common_filter.CLAIM_FROM_DT)
initial_opioid_rxs.loc[:,"DateMACombo"] = initial_opioid_rxs.MA_NUM.str.cat(initial_opioid_rxs.RX_WRITTEN_DT)
initial_opioid_rxs.loc[:,"InitInpatient"] = initial_opioid_rxs.DateMACombo.isin(common_filter.loc[common_filter.CLAIM_TYPE_CD == 1,"DateMACombo"])
initial_opioid_rxs.loc[:,"InitLongTermCare"] = initial_opioid_rxs.DateMACombo.isin(common_filter.loc[common_filter.CLAIM_TYPE_CD == 2,"DateMACombo"])
initial_opioid_rxs.loc[:,"InitOutpatient"] = initial_opioid_rxs.DateMACombo.isin(common_filter.loc[common_filter.CLAIM_TYPE_CD == 3,"DateMACombo"])
initial_opioid_rxs.loc[:,"InitPhysician"] = initial_opioid_rxs.DateMACombo.isin(common_filter.loc[common_filter.CLAIM_TYPE_CD == 4,"DateMACombo"])
initial_opioid_rxs.loc[:,"InitRehabilitation"] = initial_opioid_rxs.DateMACombo.isin(common_filter.loc[common_filter.CLAIM_TYPE_CD == 5,"DateMACombo"])
initial_opioid_rxs.loc[:,"InitHomeHealth"] = initial_opioid_rxs.DateMACombo.isin(common_filter.loc[common_filter.CLAIM_TYPE_CD == 6,"DateMACombo"])
initial_opioid_rxs.loc[:,"InitTransportAmb"] = initial_opioid_rxs.DateMACombo.isin(common_filter.loc[common_filter.CLAIM_TYPE_CD == 7,"DateMACombo"])
initial_opioid_rxs.loc[:,"InitTransportNonAmb"] = initial_opioid_rxs.DateMACombo.isin(common_filter.loc[common_filter.CLAIM_TYPE_CD == 8,"DateMACombo"])
initial_opioid_rxs.loc[:,"InitMedEquip"] = initial_opioid_rxs.DateMACombo.isin(common_filter.loc[common_filter.CLAIM_TYPE_CD == 9,"DateMACombo"])
initial_opioid_rxs.loc[:,"InitDental"] = initial_opioid_rxs.DateMACombo.isin(common_filter.loc[common_filter.CLAIM_TYPE_CD == 11,"DateMACombo"])
initial_opioid_rxs.loc[:,"InitPharmacy"] = initial_opioid_rxs.DateMACombo.isin(common_filter.loc[common_filter.CLAIM_TYPE_CD == 12,"DateMACombo"])
initial_opioid_rxs.loc[:,"InitIndipendantLab"] = initial_opioid_rxs.DateMACombo.isin(common_filter.loc[common_filter.CLAIM_TYPE_CD == 21,"DateMACombo"])
initial_opioid_rxs.loc[:,"InitIndependentClinic"] = initial_opioid_rxs.DateMACombo.isin(common_filter.loc[common_filter.CLAIM_TYPE_CD == 23,"DateMACombo"])
initial_opioid_rxs.head()


In [ ]:
institutional_filter = pd.read_csv(STATE + '/institutional_filter.csv')

In [ ]:
institutional_filter.head()

In [ ]:
institutional_filter.loc[:,"DateMACombo"] = institutional_filter.MA_NUM.str.cat(institutional_filter.DISCHARG_DT)

In [ ]:
initial_opioid_rxs.head()

In [ ]:
institutional_filter.head()

In [ ]:
Counter(institutional_filter.ADMIT_TYPE_CD.head(10000))

In [ ]:
initial_opioid_rxs.loc[:,"hasInstitutional"] = initial_opioid_rxs.DateMACombo.isin(institutional_filter.loc[:,"DateMACombo"])

In [ ]:
initial_opioid_rxs.loc[:,"InstitutionalEmergency"] = initial_opioid_rxs.DateMACombo.isin(institutional_filter.loc[institutional_filter.ADMIT_TYPE_CD == 1.0,"DateMACombo"])

In [ ]:
initial_opioid_rxs.loc[:,"InstitutionalUrgent"] = initial_opioid_rxs.DateMACombo.isin(institutional_filter.loc[institutional_filter.ADMIT_TYPE_CD == 2.0,"DateMACombo"])

In [ ]:
initial_opioid_rxs.loc[:,"InstitutionalElective"] = initial_opioid_rxs.DateMACombo.isin(institutional_filter.loc[institutional_filter.ADMIT_TYPE_CD == 3.0,"DateMACombo"])

In [ ]:
initial_opioid_rxs.loc[:,"InstitutionalNewborn"] = initial_opioid_rxs.DateMACombo.isin(institutional_filter.loc[institutional_filter.ADMIT_TYPE_CD == 4.0,"DateMACombo"])

In [ ]:
initial_opioid_rxs.loc[:,"InstitutionalTraumaCenter"] = initial_opioid_rxs.DateMACombo.isin(institutional_filter.loc[institutional_filter.ADMIT_TYPE_CD == 5.0,"DateMACombo"])

In [ ]:
initial_opioid_rxs.loc[:,"InstitutionalTraumaNotAvail"] = initial_opioid_rxs.DateMACombo.isin(institutional_filter.loc[institutional_filter.ADMIT_TYPE_CD == 9.0,"DateMACombo"])

In [ ]:
initial_opioid_rxs.hasInstitutional.mean()

# mergeproviders

In [ ]:
common_filter = common_filter.loc[common_filter.CLAIM_ID.isin(initial_opioid_rxs.CLAIM_ID), ['CLAIM_ID','CLAIM_TYPE_CD','SRVC_PROVIDER_ID','PTNT_PREG_IND','PLACE_OF_SRVC_CD']]

In [ ]:
initial_opioid_rxs = initial_opioid_rxs.merge(common_filter,how = "left", left_on = "CLAIM_ID", right_on = "CLAIM_ID")

In [ ]:
initial_opioid_rxs.groupby("SRVC_PROVIDER_ID")["MED","DAYS_SUPPLY_NUM","DISPENSE_QTY_AMT","OUTCOME"].mean().to_csv(STATE+ "/provider_averages.csv")

In [ ]:
provider_filter = pd.read_csv(STATE + '/provider_filter.csv')
provider_filter.loc[:,"PROVIDER_ID"] = provider_filter.PROVIDER_ID.astype(object)
initial_opioid_rxs = initial_opioid_rxs.merge(provider_filter, how = "left", left_on = "SRVC_PROVIDER_ID", right_on = "PROVIDER_ID")


In [ ]:
initial_opioid_rxs.head()

In [ ]:
np.mean(initial_opioid_rxs.MA_PROVIDER_ID.isna())

In [ ]:
np.mean(initial_opioid_rxs.PROVIDER_ID.isna())

### Make initial opioid features

In [ ]:
init_opioid_features = initial_opioid_rxs.loc[:,["MA_NUM", "OUTCOME", "InitInpatient", "InitLongTermCare","InitOutpatient","InitPhysician","InitRehabilitation","InitHomeHealth","InitTransportAmb","InitTransportNonAmb","InitMedEquip","InitIndependentClinic","InitDental","InitPharmacy","InitIndipendantLab","hasInstitutional","InstitutionalEmergency","InstitutionalUrgent","InstitutionalElective","InstitutionalNewborn","InstitutionalTraumaCenter","InstitutionalTraumaNotAvail"]].drop_duplicates().set_index("MA_NUM")

In [ ]:
#Creates general features about the rx's given to the patient 6 months prior

claim_id = initial_opioid_rxs.groupby('MA_NUM')['CLAIM_ID'].first()
med = initial_opioid_rxs.groupby('MA_NUM')['MED'].first()
dea_class_code = initial_opioid_rxs.groupby('MA_NUM')['DEAClassCode'].first()
longshortacting = initial_opioid_rxs.groupby('MA_NUM')['LongShortActing'].first()
generic_name = initial_opioid_rxs.groupby('MA_NUM')['Generic_Drug_Name'].first()
drug_name = initial_opioid_rxs.groupby('MA_NUM')['Drug'].first()
drug_form = initial_opioid_rxs.groupby('MA_NUM')['Master_Form'].first()
abuse_deter = initial_opioid_rxs.groupby('MA_NUM')['ABUSE_DETER'].first()

num_unique_ndcs = initial_opioid_rxs.groupby('MA_NUM')['PRODUCT_CD'].nunique()
rx_billed = initial_opioid_rxs.groupby('MA_NUM')['MA_BILLED_AMT'].first()
rx_paid = initial_opioid_rxs.groupby('MA_NUM')['MA_PAID_AMT'].first()
rx_dispense = initial_opioid_rxs.groupby('MA_NUM')['DISPENSE_QTY_AMT'].first()
rx_days = initial_opioid_rxs.groupby('MA_NUM')['DAYS_SUPPLY_NUM'].first()

#claim_common info
initial_claim_type = initial_opioid_rxs.groupby('MA_NUM')['CLAIM_TYPE_CD'].first()
initial_pregnancy = initial_opioid_rxs.groupby('MA_NUM')['PTNT_PREG_IND'].first()
initial_place_of_service = initial_opioid_rxs.groupby('MA_NUM')['PLACE_OF_SRVC_CD'].first()

#provider features for initial opioid prescription
provider_type = initial_opioid_rxs.groupby('MA_NUM')['PRVTP_RF'].first()
provider_specialty = initial_opioid_rxs.groupby('MA_NUM')['SPTCD_RF'].first()
provider_file_type = initial_opioid_rxs.groupby('MA_NUM')['PROVIDER_FILE_TYPE'].first()

In [ ]:
#Adds all these features to the feature matrix
init_opioid_features = init_opioid_features.merge(claim_id.to_frame(), left_index=True, right_index = True)
init_opioid_features = init_opioid_features.merge(med.to_frame(), left_index=True, right_index = True)
init_opioid_features = init_opioid_features.merge(dea_class_code.to_frame(), left_index=True, right_index = True)
init_opioid_features = init_opioid_features.merge(longshortacting.to_frame(), left_index=True, right_index = True)
init_opioid_features = init_opioid_features.merge(generic_name.to_frame(), left_index=True, right_index = True)
init_opioid_features = init_opioid_features.merge(drug_name.to_frame(), left_index=True, right_index = True)
init_opioid_features = init_opioid_features.merge(drug_form.to_frame(), left_index=True, right_index = True)
init_opioid_features = init_opioid_features.merge(abuse_deter.to_frame(), left_index=True, right_index = True)



init_opioid_features = init_opioid_features.merge(num_unique_ndcs.to_frame(), left_index=True, right_index = True).rename(columns = {"PRODUCT_CD":"INITIAL_RX_NDC_CODE"})
init_opioid_features = init_opioid_features.merge(rx_billed.to_frame(), left_index=True, right_index = True).rename(columns = {"MA_BILLED_AMT":"INITIAL_RX_AMOUNT_BILLED"})
init_opioid_features = init_opioid_features.merge(rx_paid.to_frame(), left_index=True, right_index = True).rename(columns = {"MA_PAID_AMT":"INITIAL_RX_AMOUNT_PAID"})
init_opioid_features = init_opioid_features.merge(rx_dispense.to_frame(), left_index=True, right_index = True).rename(columns = {"DISPENSE_QTY_AMT":"INITIAL_RX_QUANTITY"})
init_opioid_features = init_opioid_features.merge(rx_days.to_frame(), left_index=True, right_index = True).rename(columns = {"DAYS_SUPPLY_NUM":"INITIAL_RX_LENGTH"})

init_opioid_features = init_opioid_features.merge(initial_claim_type.to_frame(), left_index=True, right_index = True).rename(columns = {"CLAIM_TYPE_CD":"INITIAL_CLAIM_TYPE"})
init_opioid_features = init_opioid_features.merge(initial_pregnancy.to_frame(), left_index=True, right_index = True).rename(columns = {"PTNT_PREG_IND":"INITIAL_PREG_IND"})
init_opioid_features = init_opioid_features.merge(initial_place_of_service.to_frame(), left_index=True, right_index = True).rename(columns = {"PLACE_OF_SRVC_CD":"INITIAL_PLACE_OF_SERVICE"})


init_opioid_features = init_opioid_features.merge(provider_type.to_frame(), left_index=True, right_index = True).rename(columns = {"PRVTP_RF":"PROVIDER_TYPE"})
init_opioid_features = init_opioid_features.merge(provider_specialty.to_frame(), left_index=True, right_index = True).rename(columns = {"SPTCD_RF":"PROVIDER_SPECIALTY"})
init_opioid_features = init_opioid_features.merge(provider_file_type.to_frame(), left_index=True, right_index = True).rename(columns = {"PROVIDER_FILE_TYPE":"PROVIDER_FILE_TYPE"})

In [ ]:
init_opioid_features.head()

In [ ]:
len(init_opioid_features)

In [ ]:
init_opioid_features.to_csv(STATE + "/init_opioid_features.csv")